In [13]:
# Install the required libraries
# !pip install pystan
# !pip install prophet
# !pip install hvplot
# !pip install holoviews

In [14]:
# Imports
import pandas as pd
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
#hv.extension('bokeh')

In [11]:
from google.colab import files
uploaded = files.upload()

ModuleNotFoundError: No module named 'google.colab'

In [15]:
# Create main DataFrame from 'World Happiness Report 2005-2021' dataset.
happiness_df = pd.read_csv(Path('World Happiness Report 2005-2021.csv'), parse_dates=True, index_col='Country name')

# Set DataFrame's index to 'country_name' to prepare for concat with supplemental datasets
happiness_df.index.names=['country_name']
display(happiness_df)

/var/folders/vs/xqw7pdh1517fh7rzhwn16hhm0000gn/T/ipykernel_96738/2216956317.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  happiness_df = pd.read_csv(Path('World Happiness Report 2005-2021.csv'), parse_dates=True, index_col='Country name')


,Year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect,Confidence in national government
country_name,,,,,,,,,,,
Afghanistan,2008,3.723590,7.302574,0.450662,50.500000,0.718114,0.173169,0.881686,0.414297,0.258195,0.612072
Afghanistan,2009,4.401778,7.472446,0.552308,50.799999,0.678896,0.195469,0.850035,0.481421,0.237092,0.611545
Afghanistan,2010,4.758381,7.579183,0.539075,51.099998,0.600127,0.125859,0.706766,0.516907,0.275324,0.299357
Afghanistan,2011,3.831719,7.552006,0.521104,51.400002,0.495901,0.167723,0.731109,0.479835,0.267175,0.307386
Afghanistan,2012,3.782938,7.637953,0.520637,51.700001,0.530935,0.241247,0.775620,0.613513,0.267919,0.435440
...,...,...,...,...,...,...,...,...,...,...,...
Zimbabwe,2017,3.638300,8.241609,0.754147,52.150002,0.752826,-0.113937,0.751208,0.733641,0.224051,0.682647
Zimbabwe,2018,3.616480,8.274620,0.775388,52.625000,0.762675,-0.084747,0.844209,0.657524,0.211726,0.550508
Zimbabwe,2019,2.693523,8.196998,0.759162,53.099998,0.631908,-0.081540,0.830652,0.658434,0.235354,0.456455


In [9]:
from google.colab import files
uploaded = files.upload()

ModuleNotFoundError: No module named 'google.colab'

In [16]:
# Create supplemental DataFrame 'countries-continents', mapping countries to continent and sub-regions
# Sub-regions may represent categorical explanatory feature of happiness
regions_df = pd.read_csv(Path('countries-continents.csv'), parse_dates=True, index_col='name')

# Set supplemental DataFrame regions_df index to 'country_name' to prepare for concat with main DataFrame
regions_df.index.names=['country_name']
display(regions_df)

# regions_sub_df = regions_df['sub-region']
# display(regions_sub_df)

/var/folders/vs/xqw7pdh1517fh7rzhwn16hhm0000gn/T/ipykernel_96738/3586619326.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  regions_df = pd.read_csv(Path('countries-continents.csv'), parse_dates=True, index_col='name')


,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
country_name,,,,,,,,,,
Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,NaN,142.0,34.0,NaN
Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,NaN,150.0,154.0,NaN
Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,NaN,150.0,39.0,NaN
Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,NaN,2.0,15.0,NaN
American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,NaN,9.0,61.0,NaN
...,...,...,...,...,...,...,...,...,...,...
Wallis and Futuna,WF,WLF,876,ISO 3166-2:WF,Oceania,Polynesia,NaN,9.0,61.0,NaN
Western Sahara,EH,ESH,732,ISO 3166-2:EH,Africa,Northern Africa,NaN,2.0,15.0,NaN
Yemen,YE,YEM,887,ISO 3166-2:YE,Asia,Western Asia,NaN,142.0,145.0,NaN


In [23]:
happiness_sub_regions_df = pd.merge(happiness_df, regions_df[['sub-region']], on='country_name', how='left')
display(happiness_sub_regions_df)

,Year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect,Confidence in national government,sub-region
country_name,,,,,,,,,,,,
Afghanistan,2008,3.723590,7.302574,0.450662,50.500000,0.718114,0.173169,0.881686,0.414297,0.258195,0.612072,Southern Asia
Afghanistan,2009,4.401778,7.472446,0.552308,50.799999,0.678896,0.195469,0.850035,0.481421,0.237092,0.611545,Southern Asia
Afghanistan,2010,4.758381,7.579183,0.539075,51.099998,0.600127,0.125859,0.706766,0.516907,0.275324,0.299357,Southern Asia
Afghanistan,2011,3.831719,7.552006,0.521104,51.400002,0.495901,0.167723,0.731109,0.479835,0.267175,0.307386,Southern Asia
Afghanistan,2012,3.782938,7.637953,0.520637,51.700001,0.530935,0.241247,0.775620,0.613513,0.267919,0.435440,Southern Asia
...,...,...,...,...,...,...,...,...,...,...,...,...
Zimbabwe,2017,3.638300,8.241609,0.754147,52.150002,0.752826,-0.113937,0.751208,0.733641,0.224051,0.682647,Sub-Saharan Africa
Zimbabwe,2018,3.616480,8.274620,0.775388,52.625000,0.762675,-0.084747,0.844209,0.657524,0.211726,0.550508,Sub-Saharan Africa
Zimbabwe,2019,2.693523,8.196998,0.759162,53.099998,0.631908,-0.081540,0.830652,0.658434,0.235354,0.456455,Sub-Saharan Africa


In [16]:
# For loop developed with Ashok to circumvent concat challenges
happiness_df['sub-region'] = ""
for country_nm in happiness_df.index:
  if country_nm in regions_sub_df.index:
    happiness_df['sub-region'][country_nm] = regions_sub_df[country_nm]
display(happiness_df)

<ipython-input-16-8b4f8e151a63>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  happiness_df['sub-region'][country_nm] = regions_sub_df[country_nm]


,Year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect,Confidence in national government,Unnamed: 12,sub-region
Country name,,,,,,,,,,,,,
Afghanistan,2008,3.723590,7.302574,0.450662,50.500000,0.718114,0.173169,0.881686,0.414297,0.258195,0.612072,NaN,Southern Asia
Afghanistan,2009,4.401778,7.472446,0.552308,50.799999,0.678896,0.195469,0.850035,0.481421,0.237092,0.611545,NaN,Southern Asia
Afghanistan,2010,4.758381,7.579183,0.539075,51.099998,0.600127,0.125859,0.706766,0.516907,0.275324,0.299357,NaN,Southern Asia
Afghanistan,2011,3.831719,7.552006,0.521104,51.400002,0.495901,0.167723,0.731109,0.479835,0.267175,0.307386,NaN,Southern Asia
Afghanistan,2012,3.782938,7.637953,0.520637,51.700001,0.530935,0.241247,0.775620,0.613513,0.267919,0.435440,NaN,Southern Asia
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zimbabwe,2017,3.638300,8.241609,0.754147,52.150002,0.752826,-0.113937,0.751208,0.733641,0.224051,0.682647,3.875,Sub-Saharan Africa
Zimbabwe,2018,3.616480,8.274620,0.775388,52.625000,0.762675,-0.084747,0.844209,0.657524,0.211726,0.550508,NaN,Sub-Saharan Africa
Zimbabwe,2019,2.693523,8.196998,0.759162,53.099998,0.631908,-0.081540,0.830652,0.658434,0.235354,0.456455,NaN,Sub-Saharan Africa


In [21]:
happiness_df['sub-region'][country_nm]

Country name
Zimbabwe    Sub-Saharan Africa
Zimbabwe    Sub-Saharan Africa
Zimbabwe    Sub-Saharan Africa
Zimbabwe    Sub-Saharan Africa
Zimbabwe    Sub-Saharan Africa
Zimbabwe    Sub-Saharan Africa
Zimbabwe    Sub-Saharan Africa
Zimbabwe    Sub-Saharan Africa
Zimbabwe    Sub-Saharan Africa
Zimbabwe    Sub-Saharan Africa
Zimbabwe    Sub-Saharan Africa
Zimbabwe    Sub-Saharan Africa
Zimbabwe    Sub-Saharan Africa
Zimbabwe    Sub-Saharan Africa
Zimbabwe    Sub-Saharan Africa
Zimbabwe    Sub-Saharan Africa
Name: sub-region, dtype: object

In [23]:
happiness_df = happiness_df.drop(columns=['Unnamed: 12'])
display(happiness_df)

,Year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect,Confidence in national government,sub-region
Country name,,,,,,,,,,,,
Afghanistan,2008,3.723590,7.302574,0.450662,50.500000,0.718114,0.173169,0.881686,0.414297,0.258195,0.612072,Southern Asia
Afghanistan,2009,4.401778,7.472446,0.552308,50.799999,0.678896,0.195469,0.850035,0.481421,0.237092,0.611545,Southern Asia
Afghanistan,2010,4.758381,7.579183,0.539075,51.099998,0.600127,0.125859,0.706766,0.516907,0.275324,0.299357,Southern Asia
Afghanistan,2011,3.831719,7.552006,0.521104,51.400002,0.495901,0.167723,0.731109,0.479835,0.267175,0.307386,Southern Asia
Afghanistan,2012,3.782938,7.637953,0.520637,51.700001,0.530935,0.241247,0.775620,0.613513,0.267919,0.435440,Southern Asia
...,...,...,...,...,...,...,...,...,...,...,...,...
Zimbabwe,2017,3.638300,8.241609,0.754147,52.150002,0.752826,-0.113937,0.751208,0.733641,0.224051,0.682647,Sub-Saharan Africa
Zimbabwe,2018,3.616480,8.274620,0.775388,52.625000,0.762675,-0.084747,0.844209,0.657524,0.211726,0.550508,Sub-Saharan Africa
Zimbabwe,2019,2.693523,8.196998,0.759162,53.099998,0.631908,-0.081540,0.830652,0.658434,0.235354,0.456455,Sub-Saharan Africa


In [37]:
print(f"There are {len(happiness_df['sub-region'].unique())} sub-regions in our global happiness dataset")

There are 15 sub-regions in our global happiness dataset


In [36]:
print(f"There are {len(happiness_df.index.unique())} countries in our global happiness dataset")

There are 166 countries in our global happiness dataset


In [ ]:
We can use our sub-regions feature as a categorical predictor by encoding with OneHotEncoder.